In [330]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

import copy
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) 
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Initilizing parameters

In [278]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of input layer
    n_h -- size of hidden layer
    n_y -- size of output layer
    """
    W1 = np.random.randn(n_h, n_x)*0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y, n_h)*0.01
    b2 = np.zeros((n_y,1))


    parameters = {
        "W1":W1,
        "b1":b1,
        "W2":W2,
        "b2":b2
    }
    return parameters
    
    

## Linear forward

In [279]:
def linear_forward(A, W, b):
    """
    Arguments:
    A -- Activations from previous layer's (or input data)
    W -- weights matrix
    b -- bias vector
    """

    Z = np.dot(W, A)+b
    cache = (A, W, b)
    return Z, cache
    

## Sigmoid

In [280]:
def sigmoid(Z):
    """
    Arguments:
    Z -- value of linear function
    """
    A = 1/(1+np.exp(-Z))
    cache = Z
    return A, Z
    

## ReLU

In [281]:
def relu(Z):
    A = np.maximum(0,Z)
    cache = Z
    return A, cache

## Linear activation forward

In [282]:
def linear_activation_forward(A_prev, W, b, activation):
    """
    Arguments:
    A_prev -- actiations from previous layer (or input data)
    W -- Weights matrix
    b -- bias vector
    """
    Z , linear_cache = linear_forward(A_prev, W, b)
    
    if activation == "sigmoid":
        A, activation_cache = sigmoid(Z)
    elif activation == "relu":
        A, activation_cache = relu(Z)

    cache = (linear_cache, activation_cache)
    return A, cache

In [283]:
def L_model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters)//2
    for l in range(1, L):
        A, cache = linear_activation_forward(A, parameters["W"+str(l)], parameters["b"+str(l)], "relu")
        caches.append(cache)

    AL, cache = linear_activation_forward(A, parameters["W"+str(L)], parameters["b"+str(L)], "sigmoid")
    caches.append(cache)
    return AL, caches

## computing cost

In [284]:
def compute_cost(AL, Y):
    """
    Arguments:
    AL -- probability vectr corresponding to label prediction
    Y -- ground truth labels
    """
    m = Y.shape[1]
    cost = -1/m * np.sum(Y*np.log(AL) + (1-Y)*np.log(1-AL))
    cost = np.squeeze(cost)
    return cost

## Backward propagation

In [285]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = 1/m * np.dot(dZ, A_prev.T)
    db = 1/m * np.sum(dZ, axis = 1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)
    return dA_prev, dW, db

In [286]:
def sigmoid_backward(dA, cache):
    Z = cache
    s = 1/(1+np.exp(-Z))
    dZ =dA* s*(1-s)
    return dZ
    

In [287]:
def relu_backward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy = True)
    dZ[Z<=0] = 0
    return dZ

In [288]:
def linear_activation_backward(dA, cache, activation):
    linear_cache , activation_cache = cache
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)

    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)

    return dA_prev, dW, db
    

In [289]:
def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches)
    m = Y.shape[1]
    Y = Y.reshape(AL.shape)
    dAL =-(np.divide(Y,AL) - np.divide(1-Y, 1-AL))
    current_cache = caches[L-1]
    dA_prev, dW, db = linear_activation_backward(dAL, current_cache, "sigmoid")
    grads["dA"+str(L-1)] = dA_prev
    grads["dW"+str(L)] = dW
    grads["db"+str(L)] = db

    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev, dW, db = linear_activation_backward(grads["dA"+str(l+1)], current_cache, "relu")
        grads["dA"+str(l)] = dA_prev
        grads["dW"+str(l+1)] = dW
        grads["db"+str(l+1)] = db

    return grads

## Updating parameters

In [290]:
def update_params(grads, params, learning_rate):
    parameters = copy.deepcopy(params)
    L = len(parameters) //2

    for l in range(1,L+1):
        parameters["W"+str(l)] -= learning_rate*grads["dW"+str(l)]
        parameters["b"+str(l)] -= learning_rate*grads["db"+str(l)]

    return parameters

## Loading dataset

In [291]:
def load_data():
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes


In [292]:
train_x_orig, train_y, test_x_orig, test_y, classes = load_data()

### Normalization 

In [293]:
train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1).T
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0],-1).T

train_x = train_x_flatten/255
test_x = test_x_flatten/255

In [294]:
print("train_x shape: "+ str(train_x.shape))
print("test_x shape: " + str(test_x.shape))

train_x shape: (12288, 209)
test_x shape: (12288, 50)


## Shallow Neural Network for classification

In [323]:
def model(X, Y, layer_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost =False):
    grads = {}
    costs = []
    m = X.shape[1]
    (n_x, n_h, n_y) = layers_dims
    parameters = initialize_parameters(layer_dims)
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]

    for i in range(0,num_iterations):
        A1, cache1 = linear_activation_forward(X, W1, b1, "relu")
        A2, cache2 = linear_activation_forward(A1, W2, b2, "sigmoid")
        # print(A2, Y)

        cost = compute_cost(A2, Y)

        dA2 = -(np.divide(Y,A2)-np.divide(1-Y, 1-A2))
        dA1, dW2, db2 = linear_activation_backward(dA2, cache2, "sigmoid")
        dA0, dW1, db1 = linear_activation_backward(dA1, cache1, "relu")
        grads['dW1'] = dW1
        grads['db1'] = db1
        grads['dW2'] = dW2
        grads['db2'] = db2
        parameters = update_params(grads, parameters, learning_rate)

        W1 = parameters["W1"]
        b1 = parameters["b1"]
        W2 = parameters["W2"]
        b2 = parameters["b2"]

        if print_cost and i%100 == 0 or i == num_iterations-1:
            print(f"Cost after iteration {i}: {np.squeeze(cost)}")
        if i%100 == 0 or i == num_iterations:
            costs.append(cost)

    return parameters, costs
    

In [324]:
### CONSTANTS DEFINING THE MODEL ####
n_x = 12288     # num_px * num_px * 3
n_h = 7
n_y = 1
layers_dims = (n_x, n_h, n_y)
learning_rate = 0.0075
parameters, costs = model(train_x, train_y, layers_dims , num_iterations = 2, print_cost=False)

Cost after iteration 1: 0.6956426953726766


In [325]:
parameters, costs = model(train_x, train_y, layers_dims, num_iterations = 2500, print_cost=True)
# plot_costs(costs, learning_rate)

Cost after iteration 0: 0.6938297273399021
Cost after iteration 100: 0.6452166905106488
Cost after iteration 200: 0.6298768442088895
Cost after iteration 300: 0.5985194571901594
Cost after iteration 400: 0.5562825376952595
Cost after iteration 500: 0.5134682148083451
Cost after iteration 600: 0.4707001774665278
Cost after iteration 700: 0.43608068719484455
Cost after iteration 800: 0.3847543448592997
Cost after iteration 900: 0.3457626961164924
Cost after iteration 1000: 0.31677386925178475
Cost after iteration 1100: 0.27151084043170026
Cost after iteration 1200: 0.21456995657092695
Cost after iteration 1300: 0.19736129691940543
Cost after iteration 1400: 0.16644129915952688
Cost after iteration 1500: 0.1487234117038711
Cost after iteration 1600: 0.11606507583679075
Cost after iteration 1700: 0.10278373763116644
Cost after iteration 1800: 0.08262331630403412
Cost after iteration 1900: 0.06784743969578055
Cost after iteration 2000: 0.05730090490167227
Cost after iteration 2100: 0.050076

In [298]:
prob, caches = L_model_forward(train_x, parameters)

In [299]:
for i in range(0, prob.shape[1]):
    if prob[0,i]>0.5:
        prob[0,i] = 1
    else:
        prob[0,i] = 0

In [300]:
prob

array([[0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1.,
        0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0.,
        0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
        1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0.,
        1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
        0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1.,
        0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
        1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0.]])

In [328]:
print(f"Accuracy on train set is :{np.sum((prob==train_y)/train_x.shape[1])}")

Accuracy on train set is :0.9999999999999998


In [302]:
prob_test, caches = L_model_forward(test_x, parameters)


In [303]:
for i in range(0, prob_test.shape[1]):
    if prob_test[0,i]>0.5:
        prob_test[0,i] = 1
    else:
        prob_test[0,i] = 0

In [304]:
prob_test

array([[1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1.,
        0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1.,
        1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1.,
        1., 0.]])

In [329]:
print(f"Accuracy on test set is {np.sum((prob_test==test_y)/test_x.shape[1])}")


Accuracy on test set is 0.74
